In [1]:
import numpy as np
import pandas as pd
from funcs import linear_solve_cholesky, get_constant
import pandas_market_calendars as mcal

import numpy as np
from scipy.stats import norm
from scipy import optimize


In [2]:
# TB Page 231

A_tb = np.array([1175,1200,1225,1250,1275,1300,1325,1350,1375,1400,1425,1450,1500,1550,1575,1600])
strikes_tb = A_tb
A_tb = A_tb * -1
y_tb = np.array([178.80,154,129.05,104.20,79,54,29.05,4.25,
                 -20.40,-45.45,-70.30,-95.95,-144.70,-195,-219.80,-244.50])

call_px_tb = np.array([225.40,205.55, 186.20,167.50,149.15,131.70,115.25,99.55,84.90,
                       71.10,58.70,47.25,29.25,15.80,11.10,7.90])

put_px_tb = np.array([46.60,51.55, 57.15,63.30,70.15,77.70,86.20,95.30,105.30,116.55,129,143.20,173.95,210.80,
                     230.90,252.40])


pvf_tb = linear_solve_cholesky(y_tb, A_tb)[0]
disc_tb = linear_solve_cholesky(y_tb, A_tb)[1]

# Trading days 
nyse = mcal.get_calendar('NYSE')
schedule = nyse.schedule(start_date='2012-03-09', end_date='2012-12-22')
trading_days_tb = schedule.shape[0]

print(f"TB PVF = {pvf_tb}")
print(f"TB disc = {disc_tb}")

# PVF = Fe**-rT
# disc = e**-rt
# F = Se**(r-q)T

f_tb = pvf_tb / disc_tb
f_tb
S_tb = 1370

r_tb = np.log(pvf_tb / f_tb)/(trading_days_tb/252) * -1



q_tb = np.log(pvf_tb / 1370) / (199/252) * -1
print(f"Dividend yield is {q_tb}")
print(f"RF is {r_tb}")

TB PVF = 1349.5365684451451
TB disc = 0.9964202546130747
Dividend yield is 0.019057644308486856
RF is 0.004541278079961327


Problem 2

In [3]:
df = pd.read_excel("S_P500_ETF_Option_0917.xlsx")

# Trading days 
nyse = mcal.get_calendar('NYSE')
schedule = nyse.schedule(start_date='2017-03-16', end_date='2017-09-29')
trading_days = schedule.shape[0]

In [4]:
strikes = df['Calls'][2:].values
A = strikes * -1.0
A = A.astype(float)

mid_calls = (df['Unnamed: 2'][2:].values + df['Unnamed: 3'][2:].values)/2
mid_puts = (df['Unnamed: 7'][2:].values + df['Unnamed: 8'][2:].values)/2
y = mid_calls - mid_puts
y = y.astype(float)

In [5]:
pvf = linear_solve_cholesky(y, A)[0]
disc = linear_solve_cholesky(y, A)[1]

print(f"PVF = {pvf_tb}")
print(f"disc = {disc_tb}")
print("-"*100)
F = pvf / disc
S = 2381

r = abs(np.log(disc))/(trading_days/252)
d = abs(np.log(pvf / S) / (trading_days/252))

print(f"Using the least squared method we get the annualized dividend yield of {np.round(d, 6)} and \
risk-free rate of is {np.round(r, 6)}")

PVF = 1349.5365684451451
disc = 0.9964202546130747
----------------------------------------------------------------------------------------------------
Using the least squared method we get the annualized dividend yield of 0.016042 and risk-free rate of is 0.012516


Implied Volatility TB

In [6]:
N = norm.cdf


def c_bs(pvf, disc, K, T, sigma):
    
    d1 = np.log(pvf / (K * disc)) / sigma*np.sqrt(T) + (sigma*np.sqrt(T) / 2)
    d2 = d1 - sigma*np.sqrt(T)

    c_bs = (pvf * N(d1)) -  (K*disc*N(d2))
    return c_bs


def p_bs(pvf, disc, K, T, sigma):
    
    d1 = np.log(pvf / (K * disc)) / sigma*np.sqrt(T) + sigma*np.sqrt(T) / 2
    d2 = d1 - sigma*np.sqrt(T)

    p_bs = K*disc*N(-d2) - pvf * N(-d1)
    return p_bs

In [7]:
def get_vol(pvf, disc, K, T, mkt_val, op_type = 'c'):
    
    if op_type == 'c':
    
        def make_min(sigma):
            val = c_bs(pvf, disc, K, T, sigma)
            return val - mkt_val
    
        return optimize.newton(make_min, 0.25)
    
    elif op_type == 'p':
    
        def make_min(sigma):
            val = p_bs(pvf, disc, K, T, sigma)
            return val - mkt_val
    
        return optimize.newton(make_min, 0.25)
    
    else:
        print("Option type should be c or p")

In [8]:
T = 199/252
S = 1370
K = 1600

mkt_val = 7.90

get_vol(pvf_tb, disc_tb, K, T, mkt_val)

0.14803008304131154

In [9]:
print("Implied volatilites for call from the TB example")
print()
[get_vol(pvf_tb, disc_tb, strikes_tb[i], T, call_px_tb[i], op_type='c') for i in range(0, len(strikes_tb))]

Implied volatilites for call from the TB example



[0.25961790346850316,
 0.2513459738501823,
 0.24314011590547863,
 0.2352238764496547,
 0.2267567951834274,
 0.2188575248642874,
 0.21153936937628726,
 0.20408726543855088,
 0.19697229246552977,
 0.18960734817548644,
 0.18298794212438058,
 0.1760111711038364,
 0.16526502651887126,
 0.15429681100502524,
 0.14997745846902288,
 0.14803008304131154]

In [10]:
print("Implied volatilites for Put from the TB example")
print()
[get_vol(pvf_tb, disc_tb, strikes_tb[i], T, put_px_tb[i], op_type='p') for i in range(0, len(strikes_tb))]

Implied volatilites for Put from the TB example



[0.259462608244994,
 0.25091412557997106,
 0.24282556732761534,
 0.23478069812896052,
 0.22698450865213998,
 0.21927404685446703,
 0.2120301686671267,
 0.20433793609696693,
 0.19667704025766947,
 0.18960359044332878,
 0.18285447726225815,
 0.17751352190035216,
 0.1642721810849272,
 0.15456534803886612,
 0.14988418239879683,
 0.14700190589351142]

In [11]:
len(strikes_tb)

16

### Implied volatility Problem 2(ii)

In [12]:
df

,Calls,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Puts,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Strike,Ticker,Bid,Ask,Volm,Strike,Ticker,Bid,Ask,Volm
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2150,SPXW 9/29/17 C2150,259.299988,260.700012,0,2150,SPXW 9/29/17 P2150,34.799999,35.700001,10
3,2175,SPXW 9/29/17 C2175,238.100006,239.600006,0,2175,SPXW 9/29/17 P2175,38.5,39.400002,10
4,2200,SPXW 9/29/17 C2200,217.399994,218.899994,0,2200,SPXW 9/29/17 P2200,42.5,43.5,10
5,2225,SPXW 9/29/17 C2225,197.199997,198.699997,0,2225,SPXW 9/29/17 P2225,47.099998,48.099998,0
6,2250,SPXW 9/29/17 C2250,177.399994,178.899994,0,2250,SPXW 9/29/17 P2250,52.099998,53.200001,1
7,2275,SPXW 9/29/17 C2275,158.300003,159.699997,0,2275,SPXW 9/29/17 P2275,57.799999,58.900002,0
8,2300,SPXW 9/29/17 C2300,139.800003,141.300003,0,2300,SPXW 9/29/17 P2300,64.099998,65.300003,1
9,2325,SPXW 9/29/17 C2325,122.099998,123.599998,0,2325,SPXW 9/29/17 P2325,71.199997,72.5,2


In [13]:
print("Implied volatilites for Calls from the Problem 2(ii)")
print()
implied_vol_calls = [get_vol(pvf, disc, strikes[i], T, mid_calls[i], op_type='c') for i in range(0, len(strikes))]
implied_vol_calls

Implied volatilites for Calls from the Problem 2(ii)



[0.14535667313073308,
 0.140901751433851,
 0.1365407562301241,
 0.1322615482358161,
 0.1278391186887156,
 0.12353800404082536,
 0.11931740561410169,
 0.11513350859985298,
 0.11105951194250109,
 0.10703706179307901,
 0.10312575917500666,
 0.09938661867562867,
 0.09570959771725625,
 0.0924484818975937,
 0.08938108602611852,
 0.08437233115912572,
 0.08180906249739675,
 0.0819792399011588,
 0.08507208514160922,
 0.09776762133522705]

In [14]:
print("Implied volatilites for Puts from the Problem 2(ii)")
print()
implied_vol_puts = [get_vol(pvf, disc, strikes[i], T, mid_puts[i], op_type='p') for i in range(0, len(strikes))]
implied_vol_puts

Implied volatilites for Puts from the Problem 2(ii)



[0.14554229512753647,
 0.1411104423393879,
 0.13661885496781037,
 0.13229376512506405,
 0.1278984566176978,
 0.1236223359410501,
 0.1193615494079905,
 0.11520206369930054,
 0.11103069650548082,
 0.10703343043376758,
 0.10302727085324422,
 0.09931192963391958,
 0.09565854797933558,
 0.09229025009367883,
 0.0892401415555615,
 0.08346277581066669,
 0.08039051918641321,
 0.08331051126402218,
 0.08690434762770052,
 0.09971878148137517]

Problem 2(ii) Compare the implied volatilities of both calls and puts

In [15]:
np.array(implied_vol_calls) - np.array(implied_vol_puts)

array([-1.85621997e-04, -2.08690906e-04, -7.80987377e-05, -3.22168892e-05,
       -5.93379290e-05, -8.43319002e-05, -4.41437939e-05, -6.85550994e-05,
        2.88154370e-05,  3.63135931e-06,  9.84883218e-05,  7.46890417e-05,
        5.10497379e-05,  1.58231804e-04,  1.40944471e-04,  9.09555348e-04,
        1.41854331e-03, -1.33127136e-03, -1.83226249e-03, -1.95116015e-03])

## TODO part 3 and 4

Explicit implied volatility formula

In [16]:
def A(y):
    
    j = np.exp((1 - 2/np.pi)*y)
    k = np.exp(-1*(1 - 2/np.pi)*y)
    return (j - k)**2 

def B(y, R):
    j = 4*(np.exp((2/np.pi)*y))
    k = 2*(np.exp(-y))
    l = (np.exp(1 - (2/np.pi)*y))
    m = np.exp(-1*(1 - (2/np.pi)))
    n = (np.exp(2*y) + 1 - R**2)          
    
    return j - k * (l + m) * n 

def C(y, R):
    
    j = np.exp(-2*y)
    k = (R**2 - (np.exp(y) - 1)**2)
    l = ((np.exp(y) + 1)**2 - R**2)
    
    return j*k*l

In [36]:
C_m = 225.40
K = 1175
T = 199/252


y_ = np.log(F/K)
alpha_c = C_m/K*np.exp(-r*T)
R = 2 * alpha_c - np.exp(y_) + 1

A_ = A(y_)
B_ = B(y_, R)
C_ = C(y_, R)

beta = (2*C_) / (B_ + np.sqrt(B_**2 + 4 * A_ * C_))
lmbda = -np.pi/2*np.log(beta)


if y_ >= 0:
    C_0 = K*np.exp(-r*T) * (np.exp(y_) * A_ * (np.sqrt(2*y_) - 1/2 ))
    
    if C_m <= C_0:
        sigma_ = 1/np.sqrt(T) * (np.sqrt(lmbda + y_) - np.sqrt(lmbda - y_))

    elif C_m > C_0:
        sigma_ = 1/np.sqrt(T) * (np.sqrt(lmbda + y_) + np.sqrt(lmbda - y_))
        
if y_ < 0:
    
    C_0 = K*np.exp(-r*T) * ((np.exp(y)/2) - A_ * (-np.sqrt(-2*y_)))
    
    if C_m <= C_0:
        sigma_ = 1/np.sqrt(T) * (-np.sqrt(lmbda + y_) + np.sqrt(lmbda - y_))

    if C_m > C_0:
        sigma_ = 1/np.sqrt(T) * (np.sqrt(lmbda + y_) + np.sqrt(lmbda - y_))

    
        

/var/folders/sp/yvsgxzf94gv_51lj9l96vwgh0000gn/T/ipykernel_97043/3720225291.py:22: RuntimeWarning: invalid value encountered in sqrt
  sigma_ = 1/np.sqrt(T) * (np.sqrt(lmbda + y_) - np.sqrt(lmbda - y_))


nan

In [33]:
C_0

432.79718540486806

In [44]:
C_m = 225.40
K = 1175
T = 199/252

print(f"Cm: {C_m}")
print(f"K: {K}")
print(f"T: {T}")
print(f"F: {F}")
print(f"r: {r}")

print(f"y: {y_}")
print(f"alphaC: {alpha_c}")
print(f"R: {R}")

print(f"A: {A_}")
print(f"B: {B_}")
print(f"C: {C_}")

Cm: 225.4
K: 1175
T: 0.7896825396825397
F: 2376.4065751952007
r: 0.012516013524193846
y: 0.7043213567717433
alphaC: 0.189943142364532
R: -0.6425873962881281
A: 0.2677850151301371
B: -4.983220273238124
C: -1.3488245072385308
